In [1]:
import sys

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTContainer, LTTextBox
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [2]:
def find_textboxes_recursively(layout_obj):
    """
    再帰的にテキストボックス(LTTextBox)を探して、テキストボックスのリストを取得する
    """
    # LTTextBoxを継承するオブジェクトの場合は1要素のリストを返す。
    if isinstance(layout_obj, LTTextBox):
        return [layout_obj]
    
    # LTContainerを継承するオブジェクトは子要素を含むので、再帰的に探す。
    if isinstance(layout_obj, LTContainer):
        boxes = []
        for child in layout_obj:
            boxes.extend(find_textboxes_recursively(child))
            
        return boxes
    
    return []  # その他の場合はからリストを返す。

In [3]:
laparams = LAParams(detect_vertical=True)  # Layout Analysisのパラメータ設定。縦書の検出を有効にする。
resource_manager = PDFResourceManager()  # 共有のリソースを管理するリソースマネージャーを作成。
# ページを集めるPageAggregatorオブジェクトの作成。
device = PDFPageAggregator(resource_manager, laparams=laparams)
interpreter = PDFPageInterpreter(resource_manager, device)  # Interpreterオブジェクトの作成

In [4]:
with open(sys.argv[1], 'rb') as f:  # ファイルをバイナリ形式で開く。
    # PDFPage.get_pages()にファイルオブジェクトを指定して、PDFPageオブジェクトを順に取得する。
    # 時間がかかるファイルは、キーワード引数pagenosで処理するページ番号（0始まり）のリストを指定すると良い。
    for page in PDFPage.get_pages(f):
        interpreter.process_page(page)  # ページを処理する。
        layout = device.get_result()  # LTPageオブジェクトを取得。
        
        boxes = find_textboxes_recursively(layout)  # ページ内のテキストボックスのリストを取得する。
        # テキストボックスの左上の座標の順でテキストボックスをソートする。
        # y1(Y座標の値)は上に行くほど大きくなるので、正負を反転させている。
        boxes.sort(key=lambda b: (-b.y1, b.x0))
        
        for box in boxes:
            print('-' * 10)  # 読みやすいように区切り線を表示する。
            print(box.get_text().strip())  # テキストボックス内のテキストを表示する。

FileNotFoundError: [Errno 2] No such file or directory: '-f'

In [6]:
sys.argv?